In [ ]:
import ROOT
import ROOT
from T2KStyle.T2KStyle import GetT2K
style = GetT2K(3)
ROOT.gROOT.SetStyle("T2K")
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
path = '~/LXPLUS/DESY_testbeam/tree_v7/'
# path = '/eos/user/s/ssuvorov/DESY_testbeam/tree_v2/'
path = '~/DATA/hs_ind_freec/'
file_name = path + 's_412_430_02T'
# file_name = path + 'phi_412_45_d'
N_iter = 9

In [ ]:
def profileY(h):
    gr = ROOT.TGraphErrors()
    for i in range(1, h.GetXaxis().GetNbins()):
        temp = h.ProjectionY(f"projectionsY_bin_{i}", i, i)
        if temp.GetMaximum() < 1:
            continue
        x = h.GetXaxis().GetBinCenter(i)
        y = temp.GetBinCenter(temp.GetMaximumBin())
        gr.SetPoint(gr.GetN(), x, y)
        start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
        end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
        gr.SetPointError(gr.GetN()-1, 0., 0.5*(end-start))
    return gr

def profileX(h):
    gr = ROOT.TGraphErrors()
    for i in range(22, h.GetYaxis().GetNbins()):
        temp = h.ProjectionX(f"projectionsX_bin_{i}", i, i)
        if temp.GetMaximum() < 1:
            continue
        for j in range(temp.GetXaxis().FindBin(0.), temp.GetXaxis().GetNbins()):
            temp.SetBinContent(j, 0)
        y = h.GetYaxis().GetBinCenter(i)
        x = temp.GetBinCenter(temp.GetMaximumBin())
        gr.SetPoint(gr.GetN(), x, y)
        start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
        end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
        gr.SetPointError(gr.GetN()-1, 0.5*(end-start), 0.)
    return gr

## PRF over multiplicities

In [ ]:
PRF_function  = ROOT.TF1("PRF_function",
                         "[0]*(1+[1]*x*x + [2] * x*x*x*x) / (1+[3]*x*x+[4]*x*x*x*x)",
                         -0.027, 0.027);
PRF_function.SetLineColor(ROOT.kRed)

file = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
cl.Clear()
cl.Divide(2, 2)
for i in range(4):
    if i == 0:
        histo = file.Get('PRF_histo')
        gr = file.Get('PRF_graph')
    else:
        histo = file.Get('prf_mult/PRF_histo_' + str(i+1) + 'pad')
        gr = file.Get('prf_mult/PRF_graph_' + str(i+1) + 'pad')
    cl.cd(i+1)
#     histo.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    histo.GetYaxis().SetRangeUser(0., 1.)
    histo.Draw('colz')
#     gr.GetFunction("PRF_function").SetLineColor(ROOT.kRed)
    gr.SetLineColor(ROOT.kRed)
    gr.SetMarkerColor(ROOT.kRed)
    gr.Draw('same p')
cl.Draw()

### PRF through projection

In [ ]:
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
# hm = ROOT.TH1F('hm', '', 100, -4000., 4000.)
hm = ROOT.TH1F('hm', '', 70, 0., 70.)
file.outtree.Project('hm', 'rob_clusters', 'residual[][] > -0.5')
# hm.Fit('gaus')
c.cd()
hm.Draw('')
hm.GetXaxis().SetTitle('X_{cluster}-X_{track} [#mum]')
c.Draw()

In [ ]:
%jsroot on
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
hm = ROOT.TH2F('hm', '', 38, 0., 38., 38, 0., 38.)
file.outtree.Project('hm', 'pad_y[][]:pad_x[][]', 'qfrac[][] > 0')
c.cd()
hm.Draw('colz')
c.Draw()

In [ ]:
# %jsroot on
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
hm = ROOT.TH2F('hm', '', 38, 0., 38., 100, 0., 1.)
file.outtree.Project('hm', 'qfrac[][0]:pad_x[][0]', 'qfrac[][0] > 0')
c.cd()
hm.Draw('colz')
c.Draw()

In [ ]:
# %jsroot on
N_iter = 9
ROOT.gPad.SetGridy(0)
ROOT.gPad.SetGridx(0)
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
ROOT.gStyle.SetOptStat(0)
prf_h = ROOT.TH2F('prf_h', '', 180, -2.7, 2.7, 100, 0., 1.)
file.outtree.Project('prf_h', 'qfrac[][]:100*dx[][]', '')
c.cd()
prf_h.GetXaxis().SetTitle("X_{track} - X_{pad} [cm]")
prf_h.GetYaxis().SetTitle("Q_{pad} / Q_{cluster}")
line = ROOT.TLine(-.5, 0., -.5, 1.)
line.SetLineColor(ROOT.kRed)
line2 = ROOT.TLine(.5, 0., .5, 1.)
line2.SetLineColor(ROOT.kRed)
line3 = ROOT.TLine(1.5, 0., 1.5, 1.)
line3.SetLineColor(ROOT.kRed)
line4 = ROOT.TLine(-1.5, 0., -1.5, 1.)
line4.SetLineColor(ROOT.kRed)
line5 = ROOT.TLine(2.5, 0., 2.5, 1.)
line5.SetLineColor(ROOT.kRed)
line6 = ROOT.TLine(-2.5, 0., -2.5, 1.)
line6.SetLineColor(ROOT.kRed)
prf_h.Scale(1./1000.)
prf_h.Draw('colz')
line.Draw()
line2.Draw()
line3.Draw()
line4.Draw()
line5.Draw()
line6.Draw()
c.Draw()
c.Print('prf_sc.pdf')

In [ ]:
PRF_function  = ROOT.TF1("PRF_function",
                         "[0]*(1+[1]*x*x + [2] * x*x*x*x) / (1+[3]*x*x+[4]*x*x*x*x)",
                         -0.027, 0.027);

gr.Fit('PRF_function')
gr.GetXaxis().SetTitle("X_{track} - X_{pad} [cm]")
gr.GetYaxis().SetTitle("Q_{pad} / Q_{cluster}")
line = ROOT.TLine(-.5, 0., -.5, 1.)
line.SetLineColor(ROOT.kRed)
line2 = ROOT.TLine(.5, 0., .5, 1.)
line2.SetLineColor(ROOT.kRed)
line3 = ROOT.TLine(1.5, 0., 1.5, 1.)
line3.SetLineColor(ROOT.kRed)
line4 = ROOT.TLine(-1.5, 0., -1.5, 1.)
line4.SetLineColor(ROOT.kRed)
line5 = ROOT.TLine(2.5, 0., 2.5, 1.)
line5.SetLineColor(ROOT.kRed)
line6 = ROOT.TLine(-2.5, 0., -2.5, 1.)
line6.SetLineColor(ROOT.kRed)
gr.GetFunction('PRF_function').SetLineColor(ROOT.kRed)
gr.Draw('ap')
line.Draw()
line2.Draw()
line3.Draw()
line4.Draw()
line5.Draw()
line6.Draw()

gr.GetYaxis().SetRangeUser(0., 1.)
c.Draw()
c.Print('prf_gr.pdf')

In [ ]:
c.cd()
ROOT.gPad.SetGrid(1)
gr = profileY(prf_h)
PRF_function  = ROOT.TF1("PRF_function",
                         "[0]*(1+[1]*(x-[5])*(x-[5]) + [2] * (x-[5])*(x-[5])*(x-[5])*(x-[5])) / (1+[3]*(x-[5])*(x-[5])+[4]*(x-[5])*(x-[5])*(x-[5])*(x-[5]))",
                         -0.027, 0.027);

f = ROOT.TF1('t', '[0]*exp(-0.5*((x-[1])/[2])**2) + [3]*exp(-0.5*((x-[4])/[5])**2)', -0.02, 0.02)
f.SetParameters(0.9, 0., 0.01, 0.9, 0., 0.01)
# gr.Fit('PRF_function')
# gr.Fit('PRF_function')
# gr.Fit('PRF_function')
gr.Draw('ap')

c.Draw()

In [ ]:
%jsroot on
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
hm = ROOT.TH2F('hm', '', 720, -0.027, 0.027, 100, 0., 1.)
ha = ROOT.TH2F('ha', '', 180, -0.027, 0.027, 100, 0., 1.)
# pad size 10.2 mm
# centre 0.0051, -0.0051, -0.0102
file.outtree.Project('ha', 'qfrac[][]:dx[][]', 'qfrac[][0] > 0 && pad_x == 7')
file.outtree.Project('hm', 'qfrac[][]:dx[][]', 'qfrac[][0] > 0 && pad_x == 7 && abs(track_pos[] + 0.002) < 0.0005')
hm2 = ROOT.TH2F('hm2', '', 720, -0.027, 0.027, 100, 0., 1.)
file.outtree.Project('hm2', 'qfrac[][]:-dx[][]', 'qfrac[][0] > 0 && pad_x == 7 && abs(track_pos[] - 0.002) < 0.0005')

gr1 = profileY(ha)
bounds = 0.025
gr1.Fit('PRF_function', 'Q', '', -bounds, bounds)
gr1.Fit('PRF_function', 'Q', '', -bounds, bounds)
gr1.Fit('PRF_function', '', '', -bounds, bounds)
gr2 = profileY(hm2)
gr2.Fit('PRF_function', 'Q', '', -bounds, bounds)
gr2.Fit('PRF_function', 'Q', '', -bounds, bounds)
gr2.Fit('PRF_function', '', '', -bounds, bounds)
cw.Clear()
cw.Divide(2)
cw.cd(1)
hm.Draw('colz')
# gr1.Draw('same p')
cw.cd(1)
hm2.Draw('box same')
gr1.GetFunction('PRF_function').Draw('same')
# gr2.GetFunction('PRF_function').SetLineColor(ROOT.kRed)
# gr2.Draw('same')
cw.Draw()

### Residuals corr

In [ ]:
c.cd()
h = ROOT.TH2F('h', '', 200, -1., 1., 200, -1., 1.)
file.outtree.Project('h', '1e3*residual[5]:1e3*residual[7]')
h.Draw('colz')
print(h.GetCorrelationFactor())
c.Draw()

## Charge and time PRF

In [ ]:
%jsroot on
cl.Clear()
cl.Divide(2, 2)
N_iter = 0
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
# file = ROOT.TFile(path_loc + 'phi_200_40_iter1.root')

prf_h = ROOT.TH2F('prf_h','',  180, -0.027, 0.027, 100, 0., 1.)
prf_t = ROOT.TH2F('prf_t', '', 180, -0.027, 0.027, 100, -20., 80.)
prf_tc = ROOT.TH2F('prf_tc', '', 100, 0., 1., 100, -20., 80.)

cl.cd(1)
ROOT.gPad.SetGrid()
gr = file.Get('PRF_graph')
gr.Draw('ap')
gr.GetYaxis().SetRangeUser(0., 1.)
gr.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr.GetXaxis().SetTitle('x_{cluster} - x_{track}')

cl.cd(2)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_h', 'qfrac:dx', '')
prf_h.Draw('colz')
prf_h.GetYaxis().SetRangeUser(0., 1.)
prf_h.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
prf_h.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cl.cd(3)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t', 'time[][] - time[][0]:dx', '')
prf_t.Draw('colz')
prf_t.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_t.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cl.cd(4)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_tc', 'time[][] - time[][0]:qfrac', '')
prf_tc.Draw('colz')
prf_tc.GetYaxis().SetRangeUser(0., 1.)
prf_tc.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_tc.GetXaxis().SetTitle('Q_{pad}/Q_{cluster}')

cl.Draw()

### Time PRF

In [ ]:
cw.Clear()
cw.Divide(2)

cw.cd(1)
prf_t = ROOT.TH2F('prf_t', '', 180, -0.027, 0.027, 100, -20., 80.)
file.outtree.Project('prf_t', 'time[][] - time[][0]:dx[][]', 'dx[][] > -0.03')
prf_t.Draw('colz')
prf_t.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_t.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')
prf_t.Draw('colz')

cw.cd(2)
h = file.Get('PRF_histo_time')
h.Draw('colz')

cw.Draw()

### Profile Time PRF

In [ ]:
cw.Clear()
cw.Divide(2)
cw.cd(1)
ROOT.gPad.SetGrid()    
gr = profileY(prf_t)
gr.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
gr.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')
# gr.Fit('pol2')
gr.Draw('ap')

cw.cd(2)
ROOT.gPad.SetGrid()    
gr2 = profileX(prf_t)
gr2.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
gr2.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')
prf_t.Fit('pol2', 'Q')
prf_t.Fit('pol2', 'Q')
gr2.Fit('pol2', '', '', -0.015, -0.005)
gr2.Fit('pol2', '', '', -0.015, -0.005)
prf_t.Draw('colz')
gr2.Draw(' same p')

cw.Draw()

In [ ]:
gr2.GetFunction('pol2').GetX(30)

### Profile charge PRF

In [ ]:
cw.Clear()
cw.Divide(2)

cw.cd(1)
ROOT.gPad.SetGrid()    
gr = profileY(prf_h)
gr.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr.GetXaxis().SetTitle('x_{cluster} - x_{track}')
gr.Draw('ap')

cw.cd(2)
ROOT.gPad.SetGrid()    
gr2 = profileX(prf_h)
gr2.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr2.GetXaxis().SetTitle('x_{cluster} - x_{track}')
gr2.Draw('ap')


cw.Draw()

### Invert time PRF

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)

cw.cd(1)
prf_t1 = ROOT.TH2F('prf_t1', '', 180, -0.027, 0.027, 50, -20., 30.)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t1', 'time[][1] - time[][0]:dx[][0]', '')
prf_t1.Draw('colz')
prf_t1.GetYaxis().SetTitle('t_{2nd pad} - t_{1st pad} [bins]')
prf_t1.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cw.cd(2)
prf_t2 = ROOT.TH2F('prf_t2', '', 180, -0.027, 0.027, 100, -20., 80.)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t2', 'time[][2] - time[][0]:dx', '')
prf_t2.Draw('colz')
prf_t2.GetYaxis().SetTitle('t_{3rd pad} - t_{1st pad} [bins]')
prf_t2.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cw.Draw()

# PRF over iterations

In [ ]:
par = []
par_col = []
for i in range(5):
    par.append(ROOT.TGraphErrors())
    par_col.append(ROOT.TGraphErrors())

for it in range(N_iter):
    file = ROOT.TFile(file_name + '_iter' + str(it) + '.root')
    func = file.Get('PRF_function') 
    for i in range(5):
        par[i].SetPoint(it, it, func.GetParameter(i))
        par[i].SetPointError(it, func.GetParError(i))

# PRF over column

In [ ]:
file = ROOT.TFile(file_name + '_iter' + str(2) + '.root')
for i in range(36):
    h = file.Get(f'prf_column/PRF_histo_col_{i}')
    h.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    for j in range(5):
        par_col[j].SetPoint(i, i, PRF_function.GetParameter(j))
        par_col[j].SetPointError(i, PRF_function.GetParError(j))

In [ ]:
par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
cl.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par[i-1].GetYaxis().SetTitle(par_name[i-1])
    par[i-1].GetXaxis().SetTitle('Iteration')
    par[i-1].Draw('ap')
    
cl.Draw()

In [ ]:
c = ROOT.TCanvas('c', '', 1600, 1200)

par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
c.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par_col[i-1].GetYaxis().SetTitle(par_name[i-1])
    par_col[i-1].GetXaxis().SetTitle('Column')
    par_col[i-1].Draw('ap')
    
c.Draw()

# PRF MM vs ERAM

In [ ]:
N_iter = 5
file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_gr = file_mm.Get('PRF_graph')
PRF_er_gr = file_er.Get('PRF_graph')
PRF_mm_hi = file_mm.Get('PRF_histo')


c.Clear()
leg = ROOT.TLegend(0.2,0.2,0.4,0.4);

PRF_er_gr.GetFunction('PRF_function').SetLineColor(ROOT.kGreen)
PRF_mm_gr.GetFunction('PRF_function').SetLineColor(ROOT.kRed)
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
PRF_er_gr.GetFunction('PRF_function').GetXaxis().SetTitle('x_{pad}-x_{track}')
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetRangeUser(0., 1.)
PRF_er_gr.GetFunction('PRF_function').Draw('l')
PRF_mm_gr.GetFunction('PRF_function').Draw('same l')
leg.AddEntry(PRF_er_gr, 'ERAM2 360V', 'l')
leg.AddEntry(PRF_mm_gr, 'MM1 380V', 'l')
c.Draw()
# leg.Draw()

In [ ]:
cl.Clear()
cl.Divide(2, 2)

file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(0) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(0) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_hi = file_mm.Get('PRF_histo')
cl.cd(1)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(3)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')


file_mm_last = ROOT.TFile(folder + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er_last = ROOT.TFile(folder + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er_last.Get('PRF_histo')
PRF_mm_hi = file_mm_last.Get('PRF_histo')

cl.cd(2)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(4)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')

cl.Draw()


In [ ]:
%jsroot on
# def profileY(h):
#     gr = ROOT.TGraphErrors()
#     for i in range(1, h.GetXaxis().GetNbins()):
#         temp = h.ProjectionY(f"projectionsY_bin_{i}", i, i)
#         if temp.GetMaximum() < 1:
#             continue
#         x = h.GetXaxis().GetBinCenter(i)
#         y = temp.GetBinCenter(temp.GetMaximumBin())
#         gr.SetPoint(gr.GetN(), x, y)
#         start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
#         end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
#         gr.SetPointError(gr.GetN()-1, 0., 0.5*(end-start))
# #         print(i, x, y, 0.5*(end-start), sep='\t')
#     return gr

# file = ROOT.TFile(path_loc + '/phi/phi_412_45_d_iter0.root')
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
h = file.Get('PRF_histo')
c.cd()
h.Draw('colz')
h.GetYaxis().SetRangeUser(0., 1.)

gr = profileY(h)
# gr.Draw('same p')
func1 = ROOT.TF1('gl', "[0] * exp(-4*TMath::Log(2)*(1-[1])*TMath::Power(x/[2], 2.)) / (1+4 * [1] * TMath::Power(x/[2], 2.) )", -0.027, 0.027)
func1.SetParameters(1., 0.5, 0.005)
func2 = ROOT.TF1('prf', "[0]*(1+[1]*x*x + [2] * x*x*x*x) / (1+[3]*x*x+[4]*x*x*x*x)", -0.027, 0.027)
func2.SetParameters(0.85436010,-1919.1462,17575063.,627.88307,1.0339875e+09)
func3 = ROOT.TF1('dg', "[0] * exp(-0.5*TMath::Power(((x-[1])/[2]), 2)) + [3] * exp(-0.5*TMath::Power(((x-[4])/[5]), 2))", -0.027, 0.027)
func3.SetParameters(.7, 0., 0.005, .2, 0., 0.02)
bound = 0.026
gr.Fit('prf', '', '', -1*bound, bound)
gr.Fit('prf', '', '', -1*bound, bound)
gr.Fit('prf', '', '', -1*bound, bound)
# gr.Draw('same p')
c.Draw()

In [ ]:
gr.Fit('prf', '', '', -0.022, 0.022)
h.Draw('colz')
gr.Draw("p same")
c.Draw()

In [ ]:
# func2.SetParameters(9.07210e-01, -6.69652e+03 ,  1.83187e+07 , -5.49248e+03 ,  1.67820e+08  )
# func2.SetParameters(0.876134,  5235.38,  -2.14569e+06,  -7335.57,  6.47476e+08)
func2.SetParameters(0.903094,  -5507.41,  1.55371e+07,  -5721.22,  2.09424e+08)
func2.Draw()
c.Draw()

In [ ]:
%jsroot on
cw.Clear()
cw.Divide(2)
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
# file = ROOT.TFile(path_loc + 'phi_200_40_iter1.root')

prf_h1 = ROOT.TH2F('prf_h1','',  180, -0.027, 0.027, 100, 0., 1.)
prf_h2 = ROOT.TH2F('prf_h2','',  180, -0.027, 0.027, 100, 0., 1.)

file.outtree.Project('prf_h1', 'qfrac:dx', 'pad_charge[][0] > 900./360*(pad_charge[][1] + pad_charge[][2]) && multiplicity > 2')
file.outtree.Project('prf_h2', 'qfrac:dx', 'pad_charge[][0] < (pad_charge[][1] + pad_charge[][2]) && multiplicity > 2')

cw.cd(1)
gr1 = profileY(prf_h1)
# gr1.Fit('prf', '', '', -1*bound, bound)

prf_h1.Draw('colz')
# gr1.Draw("same")
cw.cd(2)
prf_h2.Draw('colz')
gr2 = profileY(prf_h2)
# gr2.Fit('prf', '', '', -1*bound, bound)
# gr2.Draw("same")
cw.Draw()

In [ ]:
c.cd()
h = ROOT.TH2F('h', '', 200, 0., 4000., 200, 0., 4000)
file.outtree.Project('h', 'pad_charge[][1] + pad_charge[][2]:pad_charge[][0]', 'multiplicity > 2 && pad_y[][0] == 0.5*(pad_y[][1] + pad_y[][2])')
h.GetXaxis().SetTitle('Q_{1}')
h.GetYaxis().SetTitle('Q_{2} + Q_{3}')
h.Draw('colz')

l1 = ROOT.TLine(0., 0., 4000., 4000)
l2 = ROOT.TLine(0., 0., 4000., 800)
l3 = ROOT.TLine(0., 0., 9000., 3600)
l1.Draw()
l2.Draw()
l3.Draw()
c.Draw()

## Prominence

In [ ]:
cl.Clear()
cl.Divide(2, 2)

# h = ROOT.TH2F('h', '', 100, 0., 4., 100, 0., 1.)
prf_h1 = ROOT.TH2F('prf_h1','',  180, -0.027, 0.027, 100, 0., 1.)
prf_h2 = ROOT.TH2F('prf_h2','',  180, -0.027, 0.027, 100, 0., 1.)
dx_pr = ROOT.TH2F('dx', '', 300, 0., 3., 100, -1., 1.)
h = ROOT.TH1F('h', '', 300, 0., 3)
Nclusters = 33
for entry in file.outtree:
    for i in range(1, Nclusters - 1):
        if entry.charge[i] == -999 or entry.multiplicity[i] < 2: continue
        promenence = 2 * entry.charge[i] / (entry.charge[i+1] + entry.charge[i-1])
        h.Fill(promenence)
        dx_pr.Fill(promenence, 1e3*entry.residual[i])
        for j in range(10):
            ind = i*10+j
            if entry.pad_charge[ind] < 0: continue
            
            if promenence > 1.5:
                prf_h1.Fill(entry.dx[ind], entry.qfrac[ind])
            if promenence < 0.5:
                prf_h2.Fill(entry.dx[ind], entry.qfrac[ind])
cl.cd(1)
h.GetXaxis().SetTitle('Prominence')
h.Draw()
cl.cd(2)
gr1 = profileY(prf_h1)
gr1.Fit('prf')
prf_h1.Draw('colz')
# gr1.Draw('same')
cl.cd(3)
gr2 = profileY(prf_h2)
gr2.Fit('prf')
prf_h2.Draw('colz')
# gr2.Draw('same')
cl.cd(4)
for i in range(1, dx_pr.GetXaxis().GetNbins()+1):
    tmp = dx_pr.ProjectionY(f"projectionsY_bin_{i}", i, i)
    m = tmp.GetMaximum()
    if m == 0: continue
    for j in range(1, dx_pr.GetYaxis().GetNbins()+1):
        dx_pr.SetBinContent(i, j, dx_pr.GetBinContent(i, j) / m)
dx_pr.Draw('colz')
dx_pr.GetXaxis().SetTitle('Prominence')
dx_pr.GetYaxis().SetTitle('Residual [mm]')
cl.Draw()

In [ ]:
cl.Clear()
cl.Divide(2, 2)

# h = ROOT.TH2F('h', '', 100, 0., 4., 100, 0., 1.)
# prf_h1 = ROOT.TH2F('prf_h1','',  180, -0.027, 0.027, 100, 0., 1.)
# prf_h2 = ROOT.TH2F('prf_h2','',  180, -0.027, 0.027, 100, 0., 1.)
mult_pr = ROOT.TH2F('mult_pr', '', 300, 0., 3., 10, 0., 10.)
h = ROOT.TH1F('h', '', 300, 0., 3)
Nclusters = 33
for entry in file.outtree:
    for i in range(1, Nclusters - 1):
        if entry.charge[i] == -999: continue
        promenence = 2 * entry.charge[i] / (entry.charge[i+1] + entry.charge[i-1])
        h.Fill(promenence)
        mult_pr.Fill(promenence, entry.multiplicity[i])
        
cl.cd(1)
h.GetXaxis().SetTitle('Prominence')
h.Draw()
cl.cd(2)
mult_pr.Draw('colz')
cl.Draw()        
            